In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from Bio import SeqIO
from sklearn.model_selection import train_test_split
from keras.utils import pad_sequences # padding sequences
from keras.utils import to_categorical
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding, Bidirectional, Dropout, Dense
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.regularizers import l2,l1,l1_l2
from keras.callbacks import EarlyStopping
import keras_tuner as kt
from sklearn.metrics import accuracy_score
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-5-protein-function-prediction/IA.txt
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo
/kaggle/input/train-targets-top500/train_targets_top500.npy


In [2]:
codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
def create_dict(codes):
  char_dict = {}
  for index, val in enumerate(codes):
    char_dict[val] = index+1

  return char_dict

char_dict = create_dict(codes)

print(char_dict)
print("Dict Length:", len(char_dict))

{'A': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'V': 18, 'W': 19, 'Y': 20}
Dict Length: 20


In [3]:
train_sequences = SeqIO.parse("/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta", "fasta")

In [4]:
test_sequences = SeqIO.parse("/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta", "fasta")

In [5]:
train_encode = []
for seq in train_sequences:
    row_encode = []
    for code in seq:
        row_encode.append(char_dict.get(code, 0))
    train_encode.append(np.array(row_encode))

In [6]:
test_encode = []
for seq in test_sequences:
    row_encode = []
    for code in seq:
        row_encode.append(char_dict.get(code, 0))
    test_encode.append(np.array(row_encode))

In [7]:
test_encode[1]

array([11,  3,  3, 15,  4,  3, 10, 18, 20, 14,  1,  9, 10,  1,  4, 14,  1,
        4, 15, 20,  3,  4, 11, 18,  4, 16, 11,  9,  9, 18,  1,  6, 11,  3,
       18,  4, 10, 17, 18,  4,  4, 15, 12, 10, 10, 16, 18,  1, 20,  9, 12,
       18,  8,  6,  1, 15, 15,  1, 16, 19, 15,  8,  8, 16, 16,  8,  4, 14,
        9,  4,  4, 12,  9,  6,  6,  4,  3,  9, 10,  9, 11,  8, 15,  4, 20,
       15, 14, 11, 18,  4, 17,  4, 10,  9, 10,  8,  2,  2,  3,  8, 10,  3,
       18, 10,  3,  9,  7, 10,  8, 13,  1,  1, 12, 17,  6,  4, 16,  9, 18,
        5, 20, 20,  9, 11,  9,  6,  3, 20,  7, 15, 20, 10,  1,  4,  5,  1,
       17,  6, 12,  3, 15,  9,  4,  1,  1,  4, 12, 16, 10, 18,  1, 20,  9,
        1,  1, 16,  3,  8,  1, 11, 17,  4, 10, 13, 13, 17,  7, 13,  8, 15,
       10,  6, 10,  1, 10, 12,  5, 16, 18,  5, 20, 20,  4,  8, 10, 12, 16,
       13,  3, 15,  1,  2, 15, 10,  1,  9,  1,  1,  5,  3,  3,  1,  8,  1,
        4, 10,  3, 17, 10, 16,  4,  4, 16, 20,  9,  3, 16, 17, 10,  8, 11,
       14, 10, 10, 15,  3

In [8]:
train_data_size = int(len(train_encode)/2)
train_data_size

71123

In [9]:
train_encode = train_encode[:train_data_size]

## Make length of all sequences equal 750 by padding smaller sequences and truncating larger ones

In [10]:
max_length = 750
train_pad = pad_sequences(train_encode, maxlen=max_length, padding='post', truncating='post')
train_pad.shape

(71123, 750)

In [11]:
test_pad = pad_sequences(test_encode, maxlen=max_length, padding='post', truncating='post')
test_pad.shape

(141865, 750)

In [12]:
Y = np.load('/kaggle/input/train-targets-top500/train_targets_top500.npy')

In [13]:
Y = Y[:train_data_size,:]
Y.shape

(71123, 500)

In [14]:
IX = np.arange(train_pad.shape[0])
IX_train, IX_test, _,_ = train_test_split( IX, IX, train_size=0.5, random_state=42)

In [15]:
Y[IX_train,:].shape

(35561, 500)

In [16]:
x_input = Input(shape=(max_length,))
emb = Embedding(800, 320, input_length=max_length)(x_input)
bi_rnn = Bidirectional(CuDNNLSTM(64, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.001)))(emb)
x = Dropout(0.3)(bi_rnn)
x = Dense(256,activation='relu')(x)
x_output = Dense(500, activation='sigmoid')(x)

model1 = Model(inputs=x_input, outputs=x_output)
model1.compile(optimizer=keras.optimizers.Adam(0.0001), loss=tf.nn.sigmoid_cross_entropy_with_logits, metrics=['accuracy'])

model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 750)]             0         
                                                                 
 embedding (Embedding)       (None, 750, 320)          256000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              197632    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 256)               33024     
                                                                 
 dense_1 (Dense)             (None, 500)               128500    
                                                             

In [17]:
epochs = 45
batch_size = 128

In [18]:
es = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
history1 = model1.fit(train_pad[IX_train,:], Y[IX_train,:], epochs=epochs, batch_size=batch_size,validation_split=0.1, callbacks=[es])

Epoch 1/45
251/251 [==============================] - 34s 89ms/step - loss: 2.2892 - accuracy: 0.5565 - val_loss: 1.7804 - val_accuracy: 0.6860
Epoch 2/45
251/251 [==============================] - 22s 88ms/step - loss: 1.1608 - accuracy: 0.4850 - val_loss: 1.0483 - val_accuracy: 0.6860
Epoch 4/45
251/251 [==============================] - 22s 87ms/step - loss: 0.9769 - accuracy: 0.5619 - val_loss: 0.9188 - val_accuracy: 0.6860
Epoch 5/45
251/251 [==============================] - 22s 87ms/step - loss: 0.8817 - accuracy: 0.6115 - val_loss: 0.8515 - val_accuracy: 0.6860
Epoch 6/45
251/251 [==============================] - 22s 87ms/step - loss: 0.8318 - accuracy: 0.6161 - val_loss: 0.8156 - val_accuracy: 0.6860
Epoch 7/45
251/251 [==============================] - 21s 85ms/step - loss: 0.8046 - accuracy: 0.6954 - val_loss: 0.7954 - val_accuracy: 0.6860
Epoch 8/45
251/251 [==============================] - 22s 88ms/step - loss: 0.7887 - accuracy: 0.6017 - val_loss: 0.7829 - val_accuracy:

In [25]:
Y_pred = model1.predict(train_pad[IX_test,:])

1112/1112 [==============================] - 25s 23ms/step


In [26]:
def normalize_vector(vector):
    min_val = np.min(vector)
    max_val = np.max(vector)
    normalized_vector = (vector - min_val) / (max_val - min_val)
    return normalized_vector

In [27]:
def F1_score(y_pred,y_target):
    y_pred = normalize_vector(y_pred)
    threshold = 0.5
    val_preds = np.where(y_pred> threshold, 1, 0)
    val_labels = y_target
    tp = np.sum((val_preds == 1) & (val_labels == 1))
    fp = np.sum((val_preds == 1) & (val_labels == 0))
    fn = np.sum((val_preds == 0) & (val_labels == 1))
    if tp+fp == 0:
        return 0
    precision = tp / (tp + fp)
    if tp+fn == 0:
        return 0
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)

    return f1_score

In [28]:
normalize_vector(Y_pred)

array([[9.7858191e-01, 9.7032171e-01, 9.1440189e-01, ..., 4.4406365e-06,
        5.0281469e-06, 4.2971601e-06],
       [9.7293562e-01, 9.6484977e-01, 9.0791476e-01, ..., 6.6716680e-06,
        7.4714676e-06, 6.4657365e-06],
       [9.7245139e-01, 9.6434128e-01, 9.0733850e-01, ..., 6.8708609e-06,
        7.6891793e-06, 6.6626158e-06],
       ...,
       [9.7746247e-01, 9.6917224e-01, 9.1306496e-01, ..., 4.8082625e-06,
        5.4303682e-06, 4.6585851e-06],
       [9.7361863e-01, 9.6553642e-01, 9.0871054e-01, ..., 6.3838306e-06,
        7.1572958e-06, 6.1841183e-06],
       [9.7888684e-01, 9.7063464e-01, 9.1476738e-01, ..., 4.3434738e-06,
        4.9218170e-06, 4.2017195e-06]], dtype=float32)

In [29]:
F1_score(Y_pred,Y[IX_test,:])

0.1653069267012722

In [31]:
predictions = model1.predict(test_pad)

4434/4434 [==============================] - 101s 23ms/step


In [32]:
predictions.shape

(141865, 500)

In [33]:
np.save("pred.npy", predictions)

## Manual HyperParameter Tuning

In [ ]:
num_epochs = 10
batch_sz = 256

param_grid = {
    'lstm_units': [256,512],
    'dropout_rate': [0.05,0.001],
#     'kernel_regularizer': [None, l1(0.01), l2(0.01), l1_l2(l1=0.01, l2=0.01)],
#     'recurrent_regularizer': [None, l1(0.01), l2(0.01), l1_l2(l1=0.01, l2=0.01)],
#     'bias_regularizer': [None, l1(0.01), l2(0.01), l1_l2(l1=0.01, l2=0.01)]
}

# Generate all combinations of hyperparameters
hyperparams = np.array(np.meshgrid(*param_grid.values())).T.reshape(-1, len(param_grid))

best_accuracy = 0
best_hyperparams = None

for params in hyperparams:
    lstm_units, dropout_rate,kernel_regularizer,recurrent_regularizer,bias_regularizer = params

    x_input = Input(shape=(max_length,))
    emb = Embedding(21, 128, input_length=max_length)(x_input)
    bi_rnn = Bidirectional(CuDNNLSTM(int(lstm_units), kernel_regularizer=kernel_regularizer,
                                     recurrent_regularizer=recurrent_regularizer, bias_regularizer=bias_regularizer))(emb)
    x = Dropout(dropout_rate)(bi_rnn)
    x_output = Dense(500, activation='softmax')(x)
    model = Model(inputs=x_input, outputs=x_output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    print(f"\n #### current model : {params} ### ")
    model.fit(train_pad[IX_train,:], Y[IX_train,:], epochs=num_epochs, batch_size=batch_sz,validation_split=0.1, callbacks=[es])

    # Evaluate on validation set
    val_predictions = model.predict(train_pad[IX_test,:])
    val_accuracy = accuracy_score(np.argmax(Y[IX_test,:], axis=1), np.argmax(val_predictions, axis=1))
    f1_score = F1_score(normalize_vector(val_predictions),Y[IX_test,:])
    # Check if current hyperparameters are the best so far
    print(f"model-{params}-val acc = {val_accuracy}")
    print(f"model-{params}-f1 score = {f1_score}")
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_hyperparams = params

# Print the best hyperparameters and accuracy
print("Best Hyperparameters:", best_hyperparams)
print("Best Accuracy:", best_accuracy)

## Keras Tuner for hyperparam tuning

In [ ]:
def model_builder(hp):
    x_input = Input(shape=(max_length,))
    hp_emb_input_units = hp.Int('input_dim', min_value=100, max_value=1000, step=100)
    hp_emb_output_units = hp.Int('output_dim', min_value=64, max_value=512, step=64)
    emb = Embedding(hp_emb_input_units, hp_emb_output_units, input_length=max_length)(x_input)
    hp_lstm_units = hp.Int('units', min_value=64, max_value=512, step=64)
    hp_kernel_reg = hp.Choice('kernel_regularizer', values=[1e-2,1e-3,1e-4])
    hp_recurrent_reg = hp.Choice('recurrent_regularizer', values=[1e-2,1e-3,1e-4])
    hp_bias_reg = hp.Choice('bias_regularizer', values=[1e-2,1e-3,1e-4])
    bi_rnn = Bidirectional(CuDNNLSTM(hp_lstm_units, kernel_regularizer=l2(hp_kernel_reg), recurrent_regularizer=l2(hp_recurrent_reg), bias_regularizer=l2(hp_bias_reg)))(emb)
    x = Dropout(0.1)(bi_rnn)
    hp_dense_units = hp.Int('units', min_value=64, max_value=512, step=64)
    x = Dense(512,activation='relu')(x)

    # softmax classifier
    x_output = Dense(500, activation='sigmoid')(x)

    model = Model(inputs=x_input, outputs=x_output)

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-4, 1e-5, 1e-6])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
#     model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
#                 loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#                 metrics=['accuracy'])

    return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=30,
                     factor=3,
                     directory='lstm',
                     project_name='kt')

In [ ]:
stop_early = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [ ]:
tuner.search(train_pad[IX_train,:], Y[IX_train,:], epochs=30, batch_size=128,validation_split=0.1,verbose=1, callbacks=[stop_early])

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print("Best Hyperparameters:")
for param, value in best_hps.values.items():
    print(f"{param}: {value}")

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_pad[IX_test,:], Y[IX_test,:], epochs=50, validation_split=0.1, callbacks = [stop_early])

In [ ]:
history

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(train_pad[IX_test,:])

In [ ]:
F1_score(normalize_vector(y_pred),Y[IX_test,:])

In [ ]:
Y_pred_test = model.predict(test_pad)

In [ ]:
np.save("bilstm-full-v2.npy",Y_pred_test)

LSTM v1 =>   
    Best Hyperparameters:  
    units: 147  
    kernel_regularizer: 0.01  
    recurrent_regularizer: 0.01  
    bias_regularizer: 0.01  
    learning_rate: 1e-05  
    tuner/epochs: 2  
    tuner/initial_epoch: 0  
    tuner/bracket: 2  
    tuner/round: 0  
      
F-max = 0.16

LSTM v2 =>  
    Best Hyperparameters:  
    input_dim: 210  
    output_dim: 128  
    units: 256  
    kernel_regularizer: 0.01  
    recurrent_regularizer: 0.001  
    bias_regularizer: 0.001  
    learning_rate: 1e-05  
    tuner/epochs: 2  
    tuner/initial_epoch: 0  
    tuner/bracket: 2  
    tuner/round: 0  
  
F-max = 0.16